# 논문 제목: **A Survey on Model Compression for Large Language Models**

## 저자:
- **Xunyu Zhu**¹,²
- **Jian Li**¹,²*
- **Yong Liu**³
- **Can Ma**¹,²
- **Weiping Wang**¹,²

## 소속 기관:
1. Institute of Information Engineering, Chinese Academy of Sciences  
2. School of Cyber Security, University of Chinese Academy of Sciences  
3. Gaoling School of Artificial Intelligence, Renmin University of China  

# **초록 (Abstract)**

대규모 언어 모델(LLM)은 자연어 처리 작업을 성공적으로 변혁시켰습니다. 그러나 이러한 모델의 큰 크기와 높은 계산 요구 사항은 특히 자원이 제한된 환경에서 실용적 사용에 어려움을 초래합니다. **모델 압축**은 이러한 문제를 해결하기 위한 핵심 연구 분야로 등장했습니다. 이 논문은 대규모 언어 모델(LLM)에 대한 **모델 압축 기술**에 대한 설문 조사 내용을 제공합니다.  

우리는 **양자화**, **가지치기(pruning)**, **지식 증류**와 같은 방법을 다루며, 최근의 발전 내용을 강조합니다. 또한 압축된 LLM을 평가하는 데 중요한 **벤치마킹 전략**과 **평가 지표**에 대해 논의합니다. 이 설문 조사는 연구자 및 실무자들에게 귀중한 통찰을 제공하여 LLM의 효율성과 실제 세계 적용 가능성을 향상시키고 향후 발전을 위한 기반을 마련하는 것을 목표로 합니다.

# **1 서론 (Introduction)**  

대규모 언어 모델(LLM)은 (Touvron et al., 2023a,b; Zhang et al., 2022; Scao et al., 2022; Wang and Komatsuzaki, 2021; OpenAI, 2024) 방대한 텍스트 데이터에 대해 훈련된 **Transformer 언어 모델**을 의미하며, 수십억 개 이상의 매개변수를 포함합니다. LLM은 다양한 작업에서 일관되게 뛰어난 성능을 보여주지만, 이러한 뛰어난 성능은 **대규모 크기와 높은 계산 요구사항**으로 인한 상당한 도전과제를 동반합니다.  

예를 들어, GPT-175B 모델 (Brown et al., 2020)은 **1750억 개의 매개변수**를 가지고 있으며, **FP16 (반정밀도)** 형식에서 최소 350GB의 메모리를 요구합니다. 또한 이 모델을 효율적으로 배포하려면 80GB 메모리를 갖춘 A100 GPU가 **최소 4개** 필요합니다. 이러한 문제를 해결하기 위해, **모델 압축** (Han et al., 2016)이 유용한 해결책으로 떠오르고 있습니다. 모델 압축은 대규모의 리소스 집약적 모델을 컴팩트한 버전으로 변환하여 **자원 제약 환경에서 배포**할 수 있도록 하는 방법입니다. 추가적으로 모델 압축은 LLM의 **추론 속도**를 향상시키고 **자원 효율성**을 최적화할 수 있습니다.  

이 논문에서 우리의 주된 목표는 LLM에 특화된 **모델 압축 기술** 분야에서 최근의 발전을 조명하는 것입니다. 우리의 연구는 LLM에 대한 **방법론, 메트릭 및 벤치마크**를 포괄적으로 다루는 설문 조사를 수행합니다. **그림 1**은 **LLM 모델 압축 기법**의 분류를 보여주며, 여기에는 **양자화, 가지치기(pruning), 지식 증류 및 저랭크 분해**가 포함됩니다. **그림 2**는 이러한 모델 압축 기법의 기본 **프로세스 흐름도**를 추가로 보여줍니다.  

또한, 본 연구는 이 분야에서의 **주요 과제**를 조명하고 **미래 연구 방향**을 제시합니다. 우리는 이 분야의 지속 가능한 발전을 위해 공동체 내의 협력적 노력이 필요하다고 주장하며, **생태학적으로 친화적이고 지속 가능한 LLM**을 위한 길을 열어가야 함을 강조합니다. 기존의 연구에서 신경망 모델 압축 (Li et al., 2023c)에 대한 설문은 존재했지만, LM (Rogers et al., 2020) 및 LLM (Zhao et al., 2023)에 대한 논의는 일부에 그쳤습니다. 그러나, 우리의 연구는 **LLM 모델 압축**에만 **특화된 첫 번째 설문 조사**입니다.

<img src="./images/fig_01.png" width="800">

**그림 1: 대형 언어 모델을 위한 모델 압축 방법의 분류 체계.**

<img src="./images/fig_02.png" width="800">

**그림 2: 대형 언어 모델(LLMs)을 위한 모델 압축 방법의 예시.**  
이러한 방법들 중에서 양자화 인식 학습(QAT)과 지식 증류(KD)는 특정 작업에 맞춰진 작업 기반 모델 압축 기법으로 두드러진다. 반면에, 다른 모델 압축 방법들은 작업에 구애받지 않으며, 특정 작업과 독립적으로 작동하도록 설계되었다.

# **2 메트릭 및 벤치마크**  

## **2.1 메트릭 (Metrics)**  
대규모 언어 모델(LLM)의 모델 압축 성능은 다양한 **메트릭**을 사용해 측정할 수 있으며, 이러한 메트릭은 성능의 다양한 측면을 포착합니다. 이러한 메트릭은 일반적으로 **정확도** 및 **제로샷 능력**과 함께 제공되어 LLM을 종합적으로 평가하는 데 사용됩니다.  

- **모델 크기 (Model Size)**  
  LLM의 모델 크기는 일반적으로 **총 매개변수 수**로 측정됩니다. 일반적으로 매개변수가 많을수록 훈련과 추론에 더 많은 계산 자원과 메모리를 필요로 합니다.  

- **부동 소수점 연산 (FLOPs)**  
  **FLOPs**는 **LLM의 계산 효율성**을 측정하는 지표로, LLM이 작업을 수행하기 위해 필요한 부동 소수점 연산의 수를 나타냅니다. 모델 압축에서는 FLOPs를 줄임으로써 LLM이 더 빠르고 효율적으로 실행되도록 돕습니다.  

- **평균 FLOPs 활용도 (MFU)**  
  **MFU**는 LLM이 작업 중에 계산 자원을 실제로 얼마나 효율적으로 활용하는지를 나타냅니다. MFU는 LLM이 사용한 실제 FLOPs를 장치의 최대 이론적 FLOPs와 비교하여 측정합니다. FLOPs가 LLM의 이론적 계산 요구를 평가하는 반면, **MFU**는 실제 사용된 계산 리소스의 효율성을 평가합니다.  

- **추론 시간 (Inference Time)**  
  추론 시간은 LLM이 입력 데이터를 처리하고 응답을 생성하는 데 걸리는 시간(지연 시간)을 측정합니다. 이는 실제 응용 사례에서 특히 중요한데, 사용자 질의에 실시간으로 응답하거나 대규모 데이터를 처리해야 하는 경우 유용합니다.  

- **속도 비율 (Speedup Ratio)**  
  **속도 비율**은 압축된 LLM이 원본 LLM과 비교하여 작업을 얼마나 빠르게 수행하는지를 측정합니다. 구체적으로, 압축되지 않은 모델의 추론 시간과 압축 모델의 추론 시간의 비율을 나타냅니다. 비율이 높을수록 효율성과 계산 시간이 단축되어 압축이 효과적임을 강조합니다.  

- **압축 비율 (Compression Ratio)**  
  **압축 비율**은 LLM의 크기가 압축을 통해 얼마나 줄어들었는지를 측정합니다. 이는 원본 모델 크기를 압축된 모델 크기로 나누어 계산합니다. 비율이 높을수록 더 큰 크기 감소를 나타내며, **저장 공간 및 메모리 절감**에서 압축의 효과를 보여줍니다.  

## **2.2 벤치마크와 데이터셋**  

이 벤치마크와 데이터셋의 주요 목표는 **압축된 LLM**과 압축되지 않은 모델을 비교하여 **효율성과 성능**을 측정하는 것입니다. 이러한 벤치마크와 데이터셋은 일반적으로 다양한 작업과 자연어 처리의 여러 도전 과제를 포함합니다.  

### **2.2.1 공통 벤치마크와 데이터셋**  
대부분의 연구는 잘 알려진 NLP 벤치마크와 데이터셋에서 **압축된 LLM**을 평가합니다. 예를 들어:  
- **WikiText-2** (Merity et al., 2017),  
- **C4** (Raffel et al., 2020),  
- **PTB** (Marcus et al., 1993)  

이들은 언어 모델의 **복잡도(perplexity) 성능**을 평가하기 위해 설계되었습니다.  
**LAM-BADA** (Paperno et al., 2016), **PIQA** (Tata and Patel, 2003), **OpenBookQA** (Mihaylov et al., 2018)는 언어 모델의 **제로샷 능력**을 평가하기 위해 설계되었습니다.  
**GSM8K** (Cobbe et al., 2021), **CommonsenseQA** (Talmor et al., 2019), **StrategyQA** (Geva et al., 2021)는 언어 모델의 **추론 능력**을 평가하기 위해 설계되었습니다.  

---

### **2.2.2 BIG-Bench**  
**BIG-Bench (BBH)** (Srivastava et al., 2023)는 LLM을 위해 설계된 벤치마크 세트로, **200개 이상의 NLP 작업**을 다룹니다. 예를 들어:  
- **텍스트 이해 작업(Text Comprehension Tasks)**,  
- **추론 작업(Inference Tasks)**,  
- **수학적 추론 작업(Mathematical Reasoning Tasks)**  

BBH의 목표는 다양한 복잡한 작업에서 LLM의 성능을 평가하는 것입니다. 압축된 LLM은 BBH를 사용해 다차원 작업 스펙트럼에 걸친 성능을 측정합니다.  

---

### **2.2.3 미지의 지시 데이터셋 (Unseen Instructions Datasets)**  
미지의 지시 데이터셋은 LLM이 **미지의 작업**에서 얼마나 성능을 발휘하는지를 평가하는 데 사용됩니다.  
예를 들어:  
- **Vicuna-지시 데이터셋** (Zheng et al., 2023)은 GPT-4가 생성한 데이터셋으로, **9개 카테고리**(일반, 지식 기반, 글쓰기 작업 등)에 걸친 **80개의 복잡한 질문**을 포함합니다.  
- 또 다른 데이터셋, **사용자 중심 지시 데이터셋** (Wang et al., 2023e)은 **252개의 세심하게 선정된 지시사항**으로 구성되어 있으며, Grammarly, StackOverflow, Overleaf와 같은 사용자 중심 애플리케이션에 영감을 받아 제작되었습니다.  

이러한 데이터셋은 압축된 LLM이 **새로운 작업**에 얼마나 잘 적응하고 수행할 수 있는지를 평가합니다.  

---

### **2.2.4 EleutherAI LM 하네스**  
**EleutherAI LM 하네스** (Gao et al., 2023)는 LLM을 평가하기 위한 고급 프레임워크로, **60개의 표준 학술 벤치마크**와 수백 개의 하위 작업 및 변형을 지원하는 통합 테스트 플랫폼을 제공합니다.  

이 프레임워크에서 제공되는 **표준화된 평가 작업**은 평가의 **재현성과 비교 가능성**을 보장하며, 이는 **압축된 LLM**의 **공정하고 재현 가능한 평가**를 구현하는 데 필수적입니다.  

# **3 양자화 (Quantization)**  

**양자화** (Gray and Neuhoff, 1998)는 모델의 매개변수 정밀도(비트 수)를 줄여 **추론 성능의 손실을 최소화**하는 과정을 의미합니다. 양자화는 크게 두 가지 접근법으로 분류됩니다:  
- **양자화 인식 훈련 (QAT)**  
- **사후 훈련 양자화 (PTQ)**  

이 두 접근법의 주요 차이점은 양자화 과정에서 재학습(retraining)의 필요성입니다. **PTQ**는 재학습 없이 양자화된 모델을 직접 사용할 수 있는 반면, **QAT**는 양자화에 의해 도입된 오류를 수정하기 위해 재학습이 필요합니다.  

<img src="./images/tbl_01.png" width="800">

---

## **3.1 양자화 인식 훈련 (Quantization-Aware Training)**  

**QAT**는 양자화로 인한 **성능 저하**를 보정하기 위해 양자화된 모델을 **재학습(retraining)** 하는 과정을 포함합니다.  
- 예를 들어, **LLM-QAT** (Liu et al., 2023b)는 표준 QAT 프레임워크를 LLM에 직접 적용합니다. LLM-QAT는 데이터를 생성하여 LLM 자체에서 **지식을 증류(distill)** 하고, 생성된 데이터를 기반으로 양자화된 LLM을 원본 LLM의 출력 분포와 일치하도록 학습시킵니다.  
- **BitDistiller** (Du et al., 2024)는 QAT와 **자기 증류(self-distillation)**를 결합하여 **4비트 이하** 정밀도에서 LLM 성능을 향상시킵니다. 여기에는 비대칭 양자화, 클리핑(clipping), Confidence-Aware **쿨백-라이블러 발산(Kullback-Leibler Divergence)** 목표가 포함되어 더 빠른 수렴과 우수한 결과를 제공합니다.  
- **OneBit** (Xu et al., 2024)는 혁신적인 **1비트 매개변수 표현 방법**과 효율적인 매개변수 초기화 방법을 도입하여 **1비트 양자화**를 LLM 가중치 행렬에 구현합니다. 이를 통해 극도로 낮은 비트 폭에서 LLM을 배포할 수 있는 기반을 마련합니다.  

---

**주석 1**  
_양자화 인식 훈련(QAT)은 양자화로 인한 정확도 저하를 완화할 수 있지만, 수십억 개 또는 수백억 개의 매개변수를 갖춘 LLM의 경우 재학습에는 많은 노력이 필요합니다. 실용적인 해결책은 **파라미터 효율적 미세조정 (Parameter-Efficient Fine-Tuning, PEFT)** 을 QAT 재학습 과정에 통합하는 것입니다._  

현재 QAT와 PEFT를 결합한 방법으로는:  
- **QLORA** (Dettmers et al., 2023)  
- **PEQA** (Kim et al., 2023a)  
- **LoFiQ** (Li et al., 2023a)  

이러한 방법들은 **모델 미세조정 효율성**을 목표로 하지만, 대체로 **작업 의존적**입니다.  
- **LAQ** (Jeon et al., 2024)는 LLM을 위해 LoRA 기반의 학습된 양자화 단계 크기(quantization step size)를 활용하여 **일반화**를 개선하려는 초기 시도를 진행했습니다.  

우리는 **PEFT**를 QAT 효율성 향상에 통합하는 것이 단순히 실현 가능할 뿐만 아니라 상당한 잠재력을 지니고 있다고 생각하며, 이를 심도 있게 탐구할 필요가 있습니다.

## **3.2 사후 훈련 양자화 (Post-Training Quantization, PTQ)**  

**PTQ**는 재학습 없이 **풀 프리시전 LLM**을 **저정밀도**로 변환하여 메모리와 계산 비용을 절감합니다. LLM에 대한 PTQ는 세 가지 그룹으로 분류됩니다:  
- **가중치 전용 양자화 (Weight-Only Quantization)**  
- **가중치-활성화 양자화 (Weight-Activation Quantization)**  
- **KV 캐시 양자화 (KV Cache Quantization)**  

이 세 그룹의 차이는 양자화 목표에 있습니다.  
- **가중치 전용 양자화**는 **가중치(Weights)**만 양자화하는 데 집중합니다.  
- 반면 **가중치-활성화 양자화**는 목표를 가중치와 **활성화(Activations)** 모두로 확장합니다.  

이전 연구 (Yao et al., 2023)는 활성화 양자화가 일반적으로 가중치 양자화보다 더 민감하다는 것을 나타내며, 가중치 전용 양자화를 통해 **더 낮은 비트폭**을 달성할 수 있음을 보여줍니다. 그러나 양자화된 가중치는 활성화와 곱셈하기 전에 **역양자화(dequantization)**가 필요하기 때문에 가중치 전용 양자화는 **추론 과정에서 추가적인 계산 오버헤드**를 도입하게 되며, 특정 하드웨어가 지원하는 **저비트 작업의 가속화**를 활용하지 못할 수 있습니다.  

또한, **KV 캐시 양자화**는 **KV 캐시**를 대상으로 하며, KV 캐시는 **어텐션 레이어의 키(Key)**와 **값(Value)**를 저장합니다. **KV 캐시**는 종종 많은 양의 메모리를 소모하며, 긴 토큰을 처리하는 스트림에서 **병목 현상**으로 작용할 수 있습니다. **KV 캐시 양자화**를 구현하면 처리량을 증가시키고 **긴 입력 토큰**을 보다 효율적으로 수용할 수 있습니다.

### **3.2.1 가중치 전용 양자화 (Weight-Only Quantization)**  

가중치 전용 양자화는 가장 일반적이고 널리 사용되는 방법입니다. 예를 들어, **LUT-GEMM** (Park et al., 2024)는 **이진 코딩 양자화** (BCQ) (Rastegari et al., 2016) 형식을 사용하여, LLM의 매개변수를 **이진화된 매개변수**와 **스케일링 계수 집합**으로 분해함으로써, 가중치 전용 양자화에서 **양자화된 행렬 곱셈**을 가속화합니다.  

**GPTQ** (Frantar et al., 2023)는 **최적 뇌 양자화** (Optimal Brain Quantization, OBQ) (Frantar and Alistarh, 2022)를 기반으로 한 **레이어 단위 양자화** 방법을 제안합니다. 이 방법은 **역 헤시안 정보**를 사용하여 가중치를 업데이트하고, LLM을 **3/4비트**로 양자화합니다.  

**QuIP** (Chee et al., 2023)는 가중치를 최적화하기 위해 **헤시안 행렬의 LDL 분해**를 활용합니다. 헤시안 행렬은 **검증 데이터셋**에서 무작위로 추출된 벡터를 사용해 계산되며, QuIP는 이를 크로네커 곱(Kronecker Product)과 무작위 **직교 행렬**로 조합해 헤시안 행렬과 가중치 간의 불일치성을 보장합니다. 이 두 단계를 결합해 QuIP는 **LLM을 2비트로 양자화**하면서도 성능 손실을 최소화합니다.  

---

양자화 오차를 더욱 최소화하기 위해, **가중치 전용 양자화**는 LLM에서 민감한 가중치를 식별하고 이를 **고정밀도**로 저장합니다. 이 **민감한 가중치(sensitive weights)**는 LLM의 양자화 성능에 중요한 역할을 하며, 나머지 가중치는 양자화됩니다.  

예를 들어, **AWQ** (Lin et al., 2023)는 LLM 성능에 가장 큰 영향을 미치는 **상위 1%의 가중치**를 고정밀도로 저장하고, 각 채널별 **스케일링 계수**를 식별하기 위해 **채널별 스케일링 방법**을 통합합니다. 여기서 **채널(channel)**은 모델의 개별 차원 또는 특징 맵을 의미합니다.  

**OWQ** (Lee et al., 2024)는 **활성화 이상치(activation outliers)**에 민감한 가중치를 고정밀도로 저장하고, 다른 비민감한 가중치를 양자화합니다.  

**SpQR** (Dettmers et al., 2024)는 **가중치 감도(weight sensitivity)**를 평가하기 위해 원본과 양자화된 예측 간의 **L2 오차**를 가중치 감도 지표로 사용합니다.  

---

**SqueezeLLM** (Kim et al., 2023b)는 가중치 감도를 기반으로 **가중치 군집화(weight clustering)** 알고리즘을 도입합니다. 여기서는 **k-평균 중심점(k-means centroids)**을 양자화된 가중치 값으로 사용하며, 가중치의 감도는 **헤시안 행렬**을 통해 근사됩니다. SqueezeLLM은 민감한 가중치를 **효율적인 희소 형식**으로 저장한 후, 다른 가중치를 양자화합니다.  

결과적으로 **SqueezeLLM**은 **LLM을 3비트**로 양자화하며, **FP16 기준 2배 이상의 속도 향상**을 달성합니다.

### **3.2.2 가중치-활성화 양자화 (Weight-Activation Quantization)**  

LLM의 가중치 전용 양자화와 함께, **가중치-활성화 양자화**에 집중하는 연구도 다수 존재합니다. 예를 들어:  
- **ZeroQuant** (Yao et al., 2022)는 **가중치-활성화 양자화**를 LLM에 구현한 최초의 연구로, **그룹 단위 양자화**를 통해 가중치와 활성화를 양자화하고, LLM의 가중치와 활성화를 **INT8** 정밀도로 낮춥니다.  

---

LLM의 **활성화(activations)** 에는 **이상치(outlier)** 가 존재하며, 이러한 이상치를 직접 양자화하면 LLM 성능이 크게 저하됩니다. 최근 연구들은 이러한 **이상치** 를 특별하게 처리하여 **양자화 오류** 를 줄이는 방법을 제시합니다.  

- **LLM.int8()** (Dettmers et al., 2022)는 **이상치 특징 차원**을 고정밀도로 저장하고, **벡터 단위 양자화(vector-wise quantization)** 를 사용하여 내적 행렬 곱셈의 각 항목에 대해 개별 정규화 상수를 할당해 나머지 특징들을 양자화합니다. 이를 통해 LLM.int8()은 **성능 저하 없이 8비트**로 가중치와 활성화를 양자화합니다.  

- **SmoothQuant** (Xiao et al., 2023)는 **채널별 스케일링 변환(per-channel scaling transformation)**을 설계하여 활성화 이상치를 완화합니다. 이 방법은 서로 다른 토큰이 활성화 채널 간에 **유사한 변동**을 가진다는 발견에 기반합니다.  

- **RPTQ** (Yuan et al., 2023a)는 채널 간 값의 범위가 크게 다름을 발견하고, **채널 재정렬(channel reordering)** 방식을 통합하여 채널을 **군집화**하고 재배열한 후, 각 클러스터 내의 값을 양자화하기 위해 동일한 양자화 매개변수를 사용합니다. 이 방법은 **수치 범위 차이의 영향을 완화**합니다.  

- **OliVe** (Guo et al., 2023)는 **이상치 값**이 일반 값보다 더 중요하다고 판단하고, **이상치-피해자 쌍(outlier-victim pair, OVP) 양자화**를 사용하여 하드웨어 오버헤드가 낮은 로컬 방식으로 이상치를 처리하면서 **성능 이점을 제공합니다**.  

- **OS+** (Wei et al., 2023)는 이상치가 특정 **비대칭 채널**에 집중되어 있다는 사실을 발견합니다. OS+는 **채널 단위 시프트(channel-wise shifting)**를 통해 비대칭의 영향을 제거하고, 채널 단위 스케일링을 통해 이상치의 분포를 균형화합니다.  

- **LLM-FP4** (Liu et al., 2023a)는 **부동 소수점 형식(floating-point formats)** (FP8 및 FP4)을 사용하여, 기존 정수 양자화 (INT8 및 INT4)의 한계를 극복하고 이상치를 처리합니다.  

- **LLM-FP4** (Liu et al., 2023a)는 또한 **지수 비트(exponent bits)**와 **클리핑 범위(clipping range)**가 부동 소수점 양자화의 성능에 중요한 요소임을 강조하며, **탐색 기반 프레임워크**를 도입하여 최적의 지수 비트와 최대 양자화 값을 결정합니다.  

- **OmniQuant** (Shao et al., 2024b)는 활성화 이상치를 처리하기 위해 **양자화 체인(quantization chain)**을 가중치에서 활성화로 동일하게 시프트하는 방식을 사용하고, **클리핑 임계값(clipping threshold)**을 최적화하여 가중치의 극단적인 값을 조정합니다.  

### **3.2.3 KV 캐시 양자화 (KV Cache Quantization)**  

LLM이 지원하는 입력 토큰 수가 증가함에 따라 **KV 캐시(KV Cache)**의 메모리 사용량도 증가합니다. 최근 연구들은 **KV 캐시 양자화**에 집중하여 LLM의 **메모리 사용량**을 줄이고 추론 속도를 향상시키고자 합니다.  

예를 들어:  
- **KVQuant** (Hooper et al., 2024)는 **KV 캐시 양자화**를 위한 다양한 방법을 제안합니다. 여기에는 **채널별 키 양자화(Per-Channel Key Quantization)**, **PreRoPE 키 양자화**, 그리고 **비균일 KV 캐시 양자화(Non-Uniform KV Cache Quantization)**가 포함됩니다. 이를 통해 1천만 개의 컨텍스트 길이를 가진 LLM 추론을 구현합니다.  

- **KIVI** (Liu et al., 2024)는 KV 캐시의 요소 분포를 심도 있게 분석한 후, **키 캐시(key cache)**는 채널별로 양자화하고, **값 캐시(value cache)**는 토큰별로 양자화해야 한다는 결론을 도출합니다. 최종적으로 KIVI는 **미세 조정 없이** KV 캐시를 **2비트**로 양자화하는 데 성공합니다.  

- **WKVQuant** (Yue et al., 2024)는 대규모 언어 모델(LLMs)의 양자화를 위한 혁신적인 접근법을 제시합니다. 이 방법은 과거 양자화(past-only quantization)를 통합하여 **어텐션 계산**을 정교화하고, **이차원 양자화 전략**을 사용해 **키/값(KV) 캐시의 분포를 효과적으로 관리**하며, **블록 간 재구성 정규화**를 활용해 매개변수를 최적화합니다.  

이 방법은 가중치와 KV 캐시를 모두 양자화할 수 있게 하여, **가중치-활성화 양자화와 유사한 메모리 절감**을 달성하면서도 **가중치 전용 양자화 수준에 가까운 성능**을 제공합니다.  

# **4 가지치기 (Pruning)**  

**Pruning** (LeCun et al., 1989)은 모델의 **크기나 복잡성을 줄이기 위해** 불필요한 구성 요소를 제거하는 강력한 기술입니다. **Pruning**은 다음과 같이 세 가지 범주로 나눌 수 있습니다:  
1. **비구조적 가지치기 (Unstructured Pruning)**  
2. **반구조적 가지치기 (Semi-Structured Pruning)**  
3. **구조적 가지치기 (Structured Pruning)**  

**구조적 가지치기**는 뉴런, 어텐션 헤드, 레이어와 같은 **전체 구성 요소**를 특정 규칙에 따라 제거하면서도 네트워크의 전체 구조를 유지합니다.  

반면, **비구조적 가지치기**는 개별 매개변수를 제거하여 **불규칙한 희소 구조**를 만들게 됩니다.  

**반구조적 가지치기**는 구조적 가지치기와 비구조적 가지치기의 중간에 위치하며, 세분화된 가지치기(fine-grained pruning)와 **구조적 정규화**를 동시에 달성할 수 있습니다. 특정 패턴에 따라 부분적인 매개변수를 가지치기하는 이 방법은 채널, 필터, 뉴런 전체를 제거하는 대신 세부적인 형식을 유지하며 **세분화된 구조적 가지치기** 형태를 제공합니다.  

**표 2**는 대표적인 LLM 가지치기 방법의 성능을 보여줍니다.  

<img src="./images/tbl_02.png" width="800">

## **4.1 비구조적 가지치기 (Unstructured Pruning)**  

비구조적 가지치기는 가지치기된 모델의 성능을 유지하면서 작동합니다. 따라서 LLM의 비구조적 가지치기와 관련된 연구는 종종 성능 복원을 위해 **재학습(retraining)**을 생략합니다. 그러나 비구조적 가지치기는 **불규칙한 희소 구조**를 만들어내기 때문에, 추론 가속화를 위해 **특화된 처리**나 소프트웨어 최적화가 필요합니다.  

이 분야의 혁신적 접근법 중 하나는 **SparseGPT** (Frantar and Alistarh, 2023)입니다. SparseGPT는 **원샷 가지치기 전략**을 도입하여 **재학습 없이** 실행됩니다. SparseGPT는 가지치기를 **희소 회귀 문제**로 프레임화하고 **근사 희소 회귀 솔버**를 사용해 이를 해결합니다. SparseGPT는 GPT 모델(OPT-175B 및 BLOOM-176B)의 경우 **최대 50% 이상**의 비구조적 희소성을 달성하면서도 **혼란도(perplexity)**의 최소 증가를 유지합니다.  

SparseGPT가 요구하는 가중치 업데이트 비용을 줄이기 위해 **Wanda** (Sun et al., 2024)는 **가중치의 크기**에 해당 입력 활성화(input activations)의 **노름(norm)**을 곱해 작은 값을 가지는 가중치를 가지치기하는 방법을 사용합니다. 이 방법은 **재학습**이나 가중치 업데이트 없이 모델 희소성을 달성합니다.  

가지치기에서 발생하는 오류를 최소화하면서 원하는 전체 희소성을 유지하기 위해 **SAMSP** (Shao et al., 2024a)는 가중치 행렬의 감도(sensitivity)를 평가하는 **헤시안 행렬**을 메트릭으로 활용하고, 감도에 따라 동적으로 희소성 할당을 조정합니다.  

또한, **DSnoT** (Zhang et al., 2024)는 밀집 모델과 희소 모델 간의 **재구성 오류(reconstruction error)**를 최소화하기 위해 희소 LLM 위에 **반복적 가지치기 및 성장(pruning-and-growing)** 전략을 도입하여, 높은 희소성 수준에서도 LLM 성능을 향상시킵니다.  

비구조적 가지치기를 처리하기 위한 하드웨어 지원을 제공하기 위해 **Flash-LLM** (Xia et al., 2023)은 **희소 행렬 곱셈 방법**을 도입했습니다. 이 방법은 **글로벌 메모리**에서 희소 형식의 가중치 행렬을 불러와 **고속 온칩 버퍼**에서 밀집 형식으로 재구성하여, GPU Tensor Core 하드웨어에서 고속 연산이 가능하도록 합니다.  

### **4.2 구조적 가지치기 (Structured Pruning)**  

비구조적 가지치기와 비교하여 **구조적 가지치기**는 하드웨어에 구애받지 않는 장점을 제공하며, 전통적인 하드웨어에서 가지치기 후 **추론 속도를 가속화**할 수 있습니다. 그러나 구조적 가지치기에서 더 크고 잠재적으로 중요한 구성 요소를 제거하면 **성능 저하**가 발생할 수 있으므로, 일반적으로 **효율적인 매개변수 미세 조정(fine-tuning)**이 필요합니다.  

LLM의 구조적 가지치기 연구는 **가지치기 메트릭**에 따라 다음과 같이 여러 그룹으로 나눌 수 있습니다:  
- **손실 기반 가지치기 (Loss-based Pruning)**  
- **크기 기반 가지치기 (Magnitude-based Pruning)**  
- **정규화 기반 가지치기 (Regularization-based Pruning)**  

---

**손실 기반 가지치기 (Loss-based Pruning)**  
**손실 기반 가지치기** (Molchanov et al., 2019)는 손실 또는 **기울기 정보** (예: 손실의 1차 또는 2차 도함수)가 가지치기 단위에 미치는 영향을 측정하여 해당 단위의 중요도를 평가합니다.  
- **LLM-Pruner** (Ma et al., 2023)는 **기울기 정보**를 기반으로 LLM에 대해 **원샷 구조적 가지치기**를 도입합니다. LLM-Pruner는 **종속 탐지 알고리즘**을 통해 종속 구조를 식별하고, 손실 변화에 의존하는 대신 **기울기 정보**를 사용하여 최적의 가지치기 그룹을 선택합니다. 이 접근법은 **작업에 독립적**입니다.  
- **Shortened LLaMA** (Kim et al., 2024)는 LLM-Pruner와 달리 LLM의 **너비(width)**가 아닌 **깊이(depth)**를 좁히는 것에 집중합니다. Shortened LLaMA는 **Transformer 블록**을 가지치기 가능한 단위로 선택하며, 블록의 중요도는 **손실과 그 2차 도함수**를 통해 평가됩니다. 가지치기 후, LLM-Pruner와 Shortened LLaMA는 **LoRA**를 활용하여 가지치기된 모델의 성능을 빠르게 복원합니다.  

---

**크기 기반 가지치기 (Magnitude-based Pruning)**  
**크기 기반 가지치기** (Han et al., 2015)는 가지치기 단위의 **크기(magnitude)**를 기반으로 **휴리스틱 메트릭**을 설계하여 해당 단위의 중요도를 평가합니다. 평가된 점수가 미리 정의된 임계값 아래로 떨어지는 단위는 가지치기됩니다.  
- **FLAP** (An et al., 2024)는 **구조적 변동 메트릭**을 활용하여 가지치기 적합한 **가중치 행렬의 열(column)**을 평가하고 식별합니다. 이 메트릭은 기준 값과 비교하여 입력 특징의 변동을 측정하여 가중치 열을 제거할 때의 영향을 추정합니다.  
  - FLAP는 **적응적 구조 검색(adaptive structure search)**을 통해 **글로벌 모델 압축**을 최적화하며, 가지치기 후 모델의 성능을 복원하기 위해 **기준 편향 보상 메커니즘(baseline bias compensation mechanism)**을 사용합니다. 이로 인해 **재학습(fine-tuning)**의 필요를 피할 수 있습니다.  

- **SliceGPT** (Ashkboos et al., 2024)는 Transformer 네트워크의 **계산 불변성(computational invariance)**을 활용하고, 가지치기 프로세스를 **주성분 분석(PCA)**을 통해 최적화합니다. SliceGPT는 각 레이어에서 **신호 행렬**을 주성분으로 투사하고, 변환된 가중치 행렬에서 **중요하지 않은 열이나 행**을 제거합니다. 이로써 압축된 가중치를 효율적으로 사용하면서 모델을 최적화하는 것을 목표로 합니다.  

---

**정규화 기반 가지치기 (Regularization-based Pruning)**  

**정규화 기반 가지치기** (Wen et al., 2016)는 일반적으로 **L₀, L₁, L₂ 정규화**와 같은 **정규화 항**을 손실 함수에 추가하여 LLM의 **희소성(sparsity)**을 유도합니다.  

예를 들어:  
- **Sheared LLaMA** (Xia et al., 2024)는 가지치기 마스크에 기반한 **라그랑주 승수(Lagrange multipliers)** 쌍을 사용하여 가지치기된 모델의 형태에 제약을 부과하며, 이를 **제약 최적화 문제**로 공식화합니다. 이 최적화 문제를 해결함으로써 Sheared LLaMA는 최적의 가지치기 마스크를 도출합니다.  

또한 Sheared LLaMA는 **동적 배치 로딩(dynamic batch loading)**을 도입합니다. 이 전략은 각 도메인의 **손실 감소율(loss reduction rate)**에 따라 훈련 데이터 로딩을 조정하여, 훈련 과정에서 데이터 활용 효율성을 향상시킵니다.  

---

**주석 2**  
_구조적 가지치기는 일반적으로 **불필요한 매개변수**를 제거함으로써 모델의 크기를 줄이지만, 성능 저하를 초래할 수 있습니다. 이에 대한 새로운 접근법은 **지식 증류(knowledge distillation)** (Hinton et al., 2015)와 **구조적 가지치기**를 결합하는 것입니다. 지식 증류는 LLM에서 추출한 지식을 더 작은 모델로 전달함으로써, 작은 모델이 **성능을 유지**하면서도 **크기를 줄이는** 데 도움을 줍니다._

## **4.3 반구조적 가지치기 (Semi-Structured Pruning)**  

비구조적 가지치기와 구조적 가지치기와는 별도로, **반구조적 가지치기**는 특정 패턴을 기반으로 LLM의 부분 가중치를 가지치기하는 작업을 의미합니다. 예를 들어:  
- **N:M 희소성(N:M Sparsity)**은 **M개의 연속된 요소** 중 **N개의 비영(Non-Zero) 요소**만 남기는 반구조적 가지치기의 한 예입니다.  
  - **E-Sparse** (Li et al., 2023b)는 **정보 엔트로피**를 메트릭으로 도입하여 매개변수 가중치와 입력 특징 노름의 중요도를 평가합니다. E-Sparse는 **전역 네트워크 셔플(global naive shuffle)** 및 **로컬 블록 셔플(local block shuffle)**을 통합해 정보 분포를 효율적으로 최적화하고, N:M 희소성이 LLM 정확도에 미치는 영향을 완화합니다.  

---

여러 가지치기 작업은 **반구조적 패턴**으로 일반화될 수도 있습니다. 예를 들어:  
- **SparseGPT** (Frantar and Alistarh, 2023)와 **Wanda** (Sun et al., 2024)는 모두 LLM의 N:M 희소성을 탐구합니다.  
- **SparseGPT**는 **블록 단위 가중치 분할(block-wise weight partitioning)**을 사용합니다. 각 블록은 M개의 가중치를 포함하며, **헤시안 정보**를 기반으로 재구성 오류가 가장 낮은 N개의 가중치를 선택해 가지치기합니다. 이 프로세스는 **반복적으로 가지치기 및 가중치 업데이트**를 수행하며, 모델 전체에서 원하는 희소 수준이 달성될 때까지 한 블록씩 처리됩니다.  
- **Wanda** (Sun et al., 2024)는 **구조적 N:M 가지치기**를 달성하기 위해 가중치 행렬을 M개의 연속된 가중치 그룹으로 나누고, 각 가중치의 **크기(magnitude)**와 **해당 입력 활성화(input activations)의 노름**을 곱해 **중요도 점수**를 계산합니다. 각 가중치 그룹 내에서 **점수가 가장 높은 N개의 가중치**는 유지되고 나머지는 0으로 설정되어 **구조적 N:M 가지치기**가 구현됩니다.  

---

최적의 가지치기 전략을 선택하는 것은 **대상 하드웨어와의 호환성**을 위해 중요합니다. 예를 들어:  
- **Choquette et al. (2021)**은 **Ampere Tensor Core GPU** 아키텍처(예: A100 GPU)를 소개하며, 이 하드웨어에서 **2:4 세분화된 반구조적 희소성**을 가속화하기 위한 방법을 제안합니다. 그러나 현재 Ampere 아키텍처의 구현은 **2:4 비율만** 지원하므로 다른 비율은 가속화되지 않습니다.  

---

**주석 3**  
_LLM은 종종 여러 작업에서 뛰어난 성능을 발휘하는데, 이는 다양한 작업에 대해 다수의 매개변수를 포함하기 때문입니다. **동적 가지치기** (Xia et al., 2020) 방법은 특정 작업의 요구 사항에 따라 모델의 다른 부분을 **동적으로 가지치기**하여 특정 작업에 대한 더 나은 성능을 제공합니다. 이를 통해 성능과 효율성 간의 균형을 맞출 수 있습니다._  

---

**주석 4**  
_PTQ 및 가지치기에서 **압축된 LLM의 성능**을 개선하기 위해 고품질의 **보정 데이터셋(calibration dataset)**을 준비하는 것이 중요합니다. **Williams와 Aletaras (2023)**는 보정 데이터가 모델 압축 방법에 미치는 영향을 실증적으로 연구하였으며, 보정 데이터의 선택에 따라 **다운스트림 작업의 성능**이 크게 달라질 수 있음을 발견했습니다. 고품질의 보정 데이터는 압축된 모델의 **성능과 정확도**를 향상시킬 수 있으므로, 보정 데이터의 **세심한 선택과 준비**가 필요합니다._  

# **5 지식 증류 (Knowledge Distillation)**  

**지식 증류(Knowledge Distillation, KD)** (Hinton et al., 2015)는 **대규모 복잡한 모델**(교사 모델, teacher model)의 지식을 더 작고 단순한 모델(학생 모델, student model)로 **전달**하는 기술입니다. 이 방법들은 두 가지 주요 범주로 나뉩니다 (Gu et al., 2024):  
1. **블랙박스 KD (Black-box KD)**: 교사 모델의 **출력만** 접근 가능한 경우로, 일반적으로 **폐쇄형 소스 LLM**에 해당합니다.  
2. **화이트박스 KD (White-box KD)**: 교사 모델의 **파라미터**나 **출력 분포**에 접근 가능한 경우로, 주로 **오픈소스 LLM**에 해당합니다.  

---

## **5.1 블랙박스 KD (Black-box KD)**  

블랙박스 KD는 일반적으로 **교사 LLM**이 **증류 데이터셋(distillation dataset)**을 생성하고, 이를 통해 **학생 LM**을 미세 조정(fine-tuning)함으로써 교사 LLM의 능력을 학생 LM에 **전달**하는 방식으로 작동합니다.  

- **블랙박스 KD**에서 교사 LLM으로는 **ChatGPT** (gpt-3.5-turbo) 및 **GPT-4** (OpenAI, 2024)와 같은 모델들이 주로 사용됩니다.  
- 학생 LM으로는 더 작은 언어 모델(SLMs), 예를 들어:  
   - **GPT-2** (Radford et al., 2019)  
   - **T5** (Raffel et al., 2020)  
   - **FlanT5** (Chung et al., 2024)  
   - **CodeT5** (Wang et al., 2021) 등이 널리 사용됩니다.  

연구자들은 LLM이 **창발적 능력(emergent abilities)**을 갖추고 있음을 발견했습니다. 창발적 능력이란 모델이 일정한 규모에 도달했을 때 **성능이 급격히 개선되며 나타나는 예기치 않은 능력**을 의미합니다.  

많은 **블랙박스 KD** 방법들은 **LLM의 창발적 능력**을 학생 LM으로 증류하고자 시도합니다. 우리는 주로 세 가지 **창발적 능력 증류** 방법을 소개합니다:  
1. **연쇄적 사고 증류 (Chain-of-Thought, CoT Distillation)**  
2. **맥락 내 학습 증류 (In-Context Learning, ICL Distillation)**  
3. **지시 따르기 증류 (Instruction Following, IF Distillation)**  

---

### **5.1.1 연쇄적 사고 증류 (Chain-of-Thought Distillation, CoT)**  

**CoT** (Wei et al., 2022; Wang et al., 2023c)는 LLM이 **중간 추론 단계**를 생성하도록 유도하여 복잡한 추론 작업을 **단계별로 해결**할 수 있게 합니다.  

- **Li et al. (2024b)**와 **Hsieh et al. (2023)**는 LLM을 사용해 설명을 생성하고, 다중 작업 학습 프레임워크를 활용하여 **작은 모델**의 추론 능력을 향상시키면서 **설명 생성 능력**을 강화합니다.  

- **Magister et al. (2023)**는 LLM의 **추론 능력**이 지식 증류를 통해 **작은 언어 모델(SLMs)**로 이전될 수 있음을 보여주었지만, 이는 추론 성능에서 모델과 데이터셋 크기 간 **트레이드오프**를 동반합니다.  

- **Ho et al. (2023)**는 **제로샷 CoT** 기술을 사용하여 LLM이 다양한 논리 근거를 생성하도록 유도하며, 이를 통해 학생 모델용 **증류 데이터셋**을 풍부하게 만듭니다.  

- **Shridhar et al. (2023)**는 두 가지 학생 모델(문제 분해자와 하위 문제 해결자)을 증류합니다. 여기서 **문제 분해자**는 복잡한 문제를 일련의 하위 문제로 나누고, **하위 문제 해결자**는 이러한 하위 문제를 단계별로 해결합니다.  

- **Wang et al. (2023a)**는 교사 모델이 논리적 근거를 생성하는 동안 **대조적 디코딩(contrastive decoding)**을 도입해 **지름길 문제(shortcut issues)**를 해결하고, 학생 모델 학습에서 **반사실적 추론(counterfactual reasoning)**을 도입합니다.  

- **Fu et al. (2023)**는 특정 작업 능력을 향상시키기 위해 증류를 통해 문제를 해결하면, 오히려 **일반화된 문제** 해결 성능이 저하될 수 있음을 입증하였고, 증류를 통해 **학생 모델의 수학적 능력**을 향상시키는 데 중점을 두었습니다.  

- **PaD** (Zhu et al., 2024)는 LLM이 **연쇄적 사고(CoT)** 논리 근거 대신 **사고 프로그램(Program-of-Thought, PoT)** 논리 근거를 생성하도록 유도하여 **증류 데이터셋**을 구축하고, 이 데이터셋으로 **작은 언어 모델(SLMs)**을 미세 조정합니다.  

- **Wang et al. (2023f)**는 **다중 라운드 대화형 학습 패러다임**을 확립하였으며, 이를 통해 학생 모델이 증류 과정 중 교사 모델에 피드백을 제공하여 **맞춤형 훈련 데이터**를 얻도록 합니다. 또한 **DRA**는 **자기 반성 학습 메커니즘(self-reflection learning mechanism)**을 도입하여 학생 모델이 **오류로부터 학습**하고 추론 능력을 향상시킬 수 있도록 돕습니다.  

- **Li et al. (2024c)**는 교사 모델에서 생성된 **음성 데이터(negative data)** 역시 추론 지식을 포함하고 있음을 발견하였으며, 학생 모델이 **양성 데이터(positive samples)**뿐만 아니라 **음성 데이터**에서도 학습하도록 안내합니다.  

---

### **5.1.2 맥락 내 학습 증류 (In-Context Learning Distillation, ICL)**  

**ICL** (Dong et al., 2023; Wang et al., 2023b)은 **구조화된 프롬프트**와 작업 설명, 예제를 사용하여 LLM이 **기울기 업데이트 없이** 새로운 작업을 학습하도록 돕습니다.  

- **Huang et al. (2022)**는 **맥락 내 학습 증류**라는 방법을 도입했으며, 이는 **언어 모델링 목표**와 **맥락 내 학습 목표**를 결합하여 LLM의 **맥락 내 학습 능력**을 더 작은 모델로 이전합니다. 구체적으로 이 방법은 교사 모델의 **소프트 라벨 예측**과 **하드 라벨 정답 값**을 모방함으로써 학생 모델이 다양한 작업에서 **일반화 능력**을 향상시키도록 훈련합니다.  

이 방법은 두 가지 **소수 샷 학습(few-shot learning)** 패러다임을 통합합니다:  
1. **메타 맥락 내 튜닝 (Meta-ICT)**  
2. **다중 작업 맥락 내 튜닝 (Multitask-ICT)**  

- **Meta-ICT**에서는 학생 모델이 **교사 모델의 안내**와 함께 새로운 작업에 적응합니다.  
- **Multitask-ICT**는 모든 목표 작업을 **학습 작업(training tasks)**으로 처리하여 직접 예제를 증류에 사용합니다.  

결과적으로 Multitask-ICT는 **효과적이지만**, 계산 요구사항이 더 높습니다.  

- **AICD** (Liu, 2024)는 **LLM의 자기 회귀적 특성**을 활용하여 맥락 내 **CoT(연쇄적 사고)**를 메타 학습 강제화(meta-teacher forcing)로 수행하고, 모든 맥락 내 CoT의 **우도를 최적화**함으로써, 맥락 내 학습 및 추론 능력을 작은 모델로 증류합니다.  

---

### **5.1.3 지시 따르기 증류 (Instruction Following Distillation)**  

**Instruction Following Distillation** (IF) (Ouyang et al., 2022; Brooks et al., 2023)은 **LLM의 제로샷 능력**을 강화하는 것을 목표로 하며, **프롬프트-응답 쌍**으로 이루어진 명령어 데이터셋을 사용하여 미세 조정합니다.  

- 예를 들어, **Lion** (Jiang et al., 2023)은 LLM이 **복잡한 지시사항(hard instructions)**을 식별하고 생성하도록 유도하며, 이를 통해 **학생 모델의 능력**을 향상시킵니다.  

- **LaMini-LM** (Wu et al., 2024)은 기존 명령어와 새롭게 생성된 명령어를 포함한 **2.58백만 개의 명령어**로 구성된 대규모 데이터셋을 개발하고, 이 명령어를 사용하여 다양한 모델을 미세 조정합니다.  

- **SELF-INSTRUCT** (Wang et al., 2023d)은 **학생 LMs 자체를 교사**로 사용하여 **지시 따르기 데이터셋**을 생성하고, 해당 데이터셋으로 학생 모델을 미세 조정합니다.  

- **Selective Reflection-Tuning** (Li et al., 2024a)은 교사 LLM이 기존 데이터를 **반성(reflect)**하고 개선하도록 유도하는 한편, 학생 LMs는 이 개선 사항을 평가하고 선택적으로 통합함으로써 **데이터 품질과 호환성**을 높입니다.  

---

**주석 5**  
_**블랙박스 증류**는 교사 모델의 출력을 감독으로 사용하지만, 교사 모델의 출력이 모든 입력 시나리오를 포함하지 않을 수 있습니다. 따라서 **학생 모델의 일반화 능력을 다루는 것**과 **알려지지 않은 데이터에 대한 성능 향상**은 추가적인 연구가 필요한 영역입니다._  

## **5.2 화이트박스 KD (White-box KD)**  

**화이트박스 KD**는 학생 LM이 교사 LLM의 **내부 구조**와 **지식 표현**을 더 깊이 이해하도록 하여, 종종 **더 높은 수준의 성능 개선**을 가져옵니다.  

대표적인 예로 **MINILLM** (Gu et al., 2024)이 있습니다. 이 연구는 **오픈소스 생성형 LLM**을 대상으로 증류를 연구한 최초의 작업입니다. MINILLM은 **역방향 쿨백-라이블러 발산(reverse Kullback-Leibler divergence)** 목표를 사용합니다. 이는 **생성형 언어 모델**의 KD에 더 적합하며, 학생 모델이 교사 분포의 **낮은 확률 영역**을 과대평가하는 것을 방지하고, 목표를 학습하기 위한 **효율적인 최적화 접근법**을 도출합니다.  

또한, **GKD** (Agarwal et al., 2024)는 **자가회귀 모델(auto-regressive models)**에서 증류를 탐구하며, 이 모델은 생성형 언어 모델의 하위 집합입니다. GKD는 **자가 생성 출력**을 사용해 학생 모델을 훈련하고, **교사 피드백**을 통합하며, 다양한 손실 함수를 유연하게 활용하여 학생 모델이 교사 분포를 완전히 복제할 수 없는 경우에도 유연한 증류가 가능하도록 합니다.  

위의 연구와는 달리, **TED** (Liang et al., 2023)는 **작업 인식 레이어별 증류(task-aware layer-wise distillation)** 방법을 제안합니다. 이 방법은 **작업별 필터**를 설계하여, 각 중간 레이어에서 **교사 모델과 학생 모델**의 **은닉 표현(hidden representations)**을 정렬하여, 학생과 교사 모델 간의 **지식 격차**를 줄입니다.  

---

**주석 6**  
_화이트박스 증류는 학생 LMs이 교사 LLM의 지식을 블랙박스 증류보다 더 깊이 학습하도록 하지만, 현재 **오픈소스 LLM**은 **폐쇄형 소스 모델**보다 성능이 낮아 **화이트박스 증류에서 학생 LM의 성능 향상**을 제한하고 있습니다. 이것은 화이트박스 증류 개발을 저해하는 주요 요인 중 하나입니다. 가능한 해결책은 **폐쇄형 소스 LLM에서 오픈소스 LLM으로 블랙박스 증류를 통해 지식을 이전**한 후, **화이트박스 증류**를 사용해 **오픈소스 LLM**에서 **학생 LMs**으로 지식을 이전하는 것입니다._  

---

**주석 7**  
_화이트박스 증류는 종종 LLM의 **내부 구조**를 이해하고 활용하는 것을 포함합니다. 예를 들어, **레이어 간의 연결**과 **파라미터 설정** 등이 이에 해당합니다. 레이어 간의 서로 다른 **네트워크 구조**와 상호작용을 심도 있게 탐구하면 화이트박스 증류의 **효율성**을 더욱 개선할 수 있습니다._  

# **6 저랭크 분해 (Low-Rank Factorization)**  

**저랭크 분해** (Srebro and Jaakkola, 2003)는 큰 행렬을 더 작은 행렬로 분해하여 **공간**과 **계산 비용**을 절감하는 방법입니다. 예를 들어, 큰 행렬 $ W $를 두 개의 작은 행렬 $ U $와 $ V $로 분해합니다. ($ W \approx UV $), 여기서 $ U $는 $ m \times k $ 행렬이고, $ V $는 $ k \times n $ 행렬이며, $ k $는 $ m $과 $ n $보다 훨씬 작습니다.  

최근 연구들은 LLM을 압축하기 위해 저랭크 분해를 활용하며, 이 분야에서 상당한 성공을 거두고 있습니다.  
- **LPLR** (Saha et al., 2023)는 **무작위 저랭크 및 저정밀 분해(randomized low-rank and low-precision factorization)**를 통해 LLM의 가중치 행렬을 압축합니다. 구체적으로, LPLR은 무작위 스케치 기법을 사용해 행렬의 **공간을 근사**하고, 이 열(columns)을 양자화된 공간에 투사하여 **저정밀도**로 저장된 저랭크 요인을 생성합니다.  

- **ASVD** (Yuan et al., 2023b)는 **활성화 분포**가 압축 성능에 영향을 미친다는 사실을 발견했습니다. 이를 해결하기 위해 ASVD는 가중치 행렬을 **대각 행렬**로 스케일링하여 입력 특징 채널의 활성화 분포와 일치하는 **스케일링 계수**를 포함시킵니다. 또한, ASVD는 각 레이어의 가중치 행렬에서 **특이값 분포**를 분석하여 **가장 적합한 압축 비율**을 할당해 압축 과정 중 모델 성능 손실을 최소화합니다.  

- **Sharma et al. (2024)**는 LLM 성능이 **레이어 선택적 랭크 감소 (Layer-Selective Rank Reduction, LASER)**를 Transformer 모델의 특정 레이어에 적용함으로써 크게 개선될 수 있음을 입증했습니다. LASER는 가중치 행렬의 **고차원 성분**을 선택적으로 줄임으로써 **희소한 훈련 데이터**를 처리하는 모델의 능력과 질문 **재구성** 저항성을 향상시킵니다.  

# **7 과제와 미래 방향 (Challenges and Future Directions)**  

## **7.1 더 발전된 방법들 (More Advanced Methods)**  
LLM에 대한 **모델 압축 기술** 연구는 여전히 초기 단계에 있습니다. 이러한 압축된 LLM은 기존 연구들 (Frantar and Alistarh, 2023; Liu et al., 2023b; Ho et al., 2023)에서 입증된 것처럼, 여전히 압축되지 않은 모델에 비해 **성능 격차**를 보이고 있습니다. LLM에 특화된 더 발전된 모델 압축 방법을 심도 있게 탐구함으로써, 비압축 모델의 성능을 향상시킬 잠재력이 있습니다.  

---

## **7.2 다른 모델에서 모델 압축 방법 확장하기 (Scaling up Model Compression Methods from Other Models)**  
우리 논문에서는 LLM에 대해 여러 대표적인 모델 압축 방법을 소개합니다. 그러나 기존의 많은 **전통적인 소형 모델** 압축 기법들은 여전히 널리 사용되고 있습니다. 예를 들어:  
- **로터리 티켓 (Lottery Tickets)** (Frankle and Carbin, 2019)  
- **파라미터 공유 (Parameter Sharing)** (Savarese and Maire, 2019)  

이들은 **소형 모델**에서 주로 사용되는 모델 압축 방법입니다. 이러한 방법들은 **LLM 시대**에서도 상당한 잠재력을 지니고 있습니다. 앞으로의 연구는 이러한 압축 방법들이 **LLM에 어떻게 확장될 수 있는지**를 탐구하여 더 나은 압축 성능을 달성하는 데 초점을 맞춰야 합니다.  

### **7.3 LLM 추론 및 배포 (LLM Inference and Deployment)**  

압축된 LLM의 **배포 효율성**은 탐구해야 할 중요한 영역입니다. 여기에는 **연산 집약도(arithmetic intensity)**, **메모리 크기**, **처리량(throughput)**을 포함한 여러 평가 지표가 활용됩니다.  

또한, **Roofline 모델** (Williams et al., 2009)과 같은 분석 도구를 사용하여 특정 하드웨어에서 압축된 LLM의 **리소스 효율성**을 평가할 수 있습니다. 압축된 LLM의 배포 효율성을 평가하면 연구자들이 **다양한 모델 압축 방법의 장단점**을 선택하고 분석하는 데 도움을 주며, 이 방법들을 더 최적화하는 데 기여합니다.  

---

## **7.4 스케일링 법칙의 영향 (The Effect of Scaling Law)**  

**스케일링 법칙** (Kaplan et al., 2020)은 모델 크기, 데이터셋 크기, 계산 자원이 LLM의 성능에 미치는 **중대한 영향**을 강조합니다. 그러나 스케일링 법칙은 압축된 LLM에서 중요한 도전 과제를 제시합니다. 즉, **모델 크기와 성능 간의 트레이드오프**가 발생합니다.  

스케일링 법칙을 뒷받침하는 메커니즘과 이론을 심층적으로 탐구하는 것은 이러한 한계를 이해하고 잠재적으로 극복하기 위해 중요합니다.  

---

## **7.5 LLM 압축을 위한 AutoML (AutoML for LLM Compression)**  

기존의 압축 기술은 상당한 발전을 이루었지만 여전히 **수작업(manual design)**에 크게 의존하고 있습니다. 예를 들어, **지식 증류(knowledge distillation)**에 적합한 학생 모델 아키텍처를 설계하는 데 많은 인적 노력이 필요합니다.  

이러한 수작업 의존성을 줄이기 위해 **자동화된 머신러닝 (AutoML)** 기술을 모델 압축과 결합하는 것이 실현 가능한 해결책입니다.  
- 예를 들어, **메타 학습 (Meta-Learning)** (Finn et al., 2017) 및 **신경 아키텍처 검색 (Neural Architecture Search, NAS)** (Zoph and Le, 2017)를 모델 압축과 결합하면 **하이퍼파라미터를 자동으로 선택**하고, 압축된 모델의 아키텍처와 규모를 맞춤화할 수 있습니다. 이를 통해 **인간 개입을 최소화**하고 **비용을 절감**할 수 있습니다.  

또한, AutoML은 특정 작업 요구사항에 맞춘 **최적의 모델 압축 전략**을 식별하여, 성능 저하 없이 **압축률**을 높일 수 있습니다.  

---

## **7.6 LLM 압축의 설명 가능성 (Explainability of LLM Compression)**  

이전 연구 (Stanton et al., 2021; Xu et al., 2021)는 **사전 훈련된 언어 모델(PLMs)**에 적용된 **모델 압축 기술의 설명 가능성**에 대한 중대한 우려를 제기했습니다. 이러한 문제들은 **LLM 압축 방법**에도 동일하게 적용됩니다.  

예를 들어, **CoT 증류**는 작은 언어 모델(SLM)의 추론 성능을 향상시킬 수 있지만, 그 과정에서 CoT 능력이 **어떻게 전달되는지**에 대한 메커니즘은 여전히 불분명합니다.  

이 도전 과제는 **모델 압축 접근법과 설명 가능성**을 통합하는 것이 중요함을 강조합니다. 설명 가능성은 **변화 및 트레이드오프**를 명확히 할 뿐만 아니라 **효율성과 정확성**을 향상시키는 데 기여합니다.  

또한, 모델의 **해석 가능성**은 압축된 모델의 성능을 평가하는 데 도움을 주어, 실제 응용 사례에 맞는 결과를 보장합니다.  

# **8 결론 (Conclusion)**  

이 설문에서 우리는 **대규모 언어 모델(LLMs)의 모델 압축 기술**을 탐구했습니다. 우리의 논의는 **압축 방법, 메트릭 및 벤치마크 데이터셋**에 걸쳐 이루어졌습니다. LLM 압축을 심층적으로 다룸으로써 우리는 이 기술의 **도전 과제와 기회**를 강조했습니다.  

이 설문은 **가치 있는 참고자료**가 되어 현재의 연구 동향에 대한 통찰을 제공하고, 이 중요한 주제에 대한 **지속적인 탐구**를 촉진하는 것을 목표로 합니다.  

---

# **감사 인사 (Acknowledgments)**  

우리는 귀중한 피드백과 논의를 제공해 주신 **익명의 리뷰어**와 **액션 에디터**께 감사드립니다. Jian Li의 연구는 **중국 국가 자연과학 재단** (No. 62106257)의 일부 지원을 받았습니다.  

Yong Liu의 연구는 **중국 국가 자연과학 재단** (No. 62076234), **베이징 우수 청년 과학자 프로그램** (No. BJWZYJH012019100020098), **Unicom 혁신 생태 협력 계획**, 및 **CCF-Huawei Populus Grove Fund**의 일부 지원을 받았습니다.  

# **참고문헌 (References)**  

1. **Rishabh Agarwal, Nino Vieillard, Yongchao Zhou, Piotr Stanczyk, Sabela Ramos Garea, Matthieu Geist, and Olivier Bachem. 2024.**  
   _Generalized knowledge distillation for auto-regressive language models._  
   **In The Twelfth International Conference on Learning Representations.**  

2. **Yongqi An, Xu Zhao, Tao Yu, Ming Tang, and Jinqiao Wang. 2024.**  
   _Fluctuation-based adaptive structured pruning for large language models._  
   **In Thirty-Eighth AAAI Conference on Artificial Intelligence, AAAI 2024, Thirty-Sixth Conference on Innovative Applications of Artificial Intelligence, IAAI 2024, Fourteenth Symposium on Educational Advances in Artificial Intelligence, EAAI 2014, February 20-27, 2024, Vancouver, Canada,** pages 10865–10873. AAAI Press.  

3. **Saleh Ashkboos, Maximilian L. Croci, Marcelo Gennari do Nascimento, Torsten Hoefler, and James Hensman. 2024.**  
   _SliceGPT: Compress large language models by deleting rows and columns._  
   **In The Twelfth International Conference on Learning Representations.**  

4. **Tim Brooks, Aleksander Holynski, and Alexei A. Efros. 2023.**  
   _Instructpix2pix: Learning to follow image editing instructions._  
   **In IEEE/CVF Conference on Computer Vision and Pattern Recognition, CVPR 2023, Vancouver, BC, Canada, June 17-24, 2023,** pages 18392–18402. IEEE.  

5. **Tom B. Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared Kaplan, Prafulla Dhariwal, Arvind Neelakantan, Pranav Shyam, Girish Sastry, Amanda Askell, Sandhini Agarwal, Ariel Herbert-Voss, Gretchen Krueger, Tom Henighan, Rewon Child, Aditya Ramesh, Daniel M. Ziegler, Jeffrey Wu, Clemens Winter, Christopher Hesse, Mark Chen, Eric Sigler, Mateusz Litwin, Scott Gray, Benjamin Chess, Jack Clark, Christopher Berner, Sam McCandlish, Alec Radford, Ilya Sutskever, and Dario Amodei. 2020.**  
   _Language models are few-shot learners._  
   **In Advances in Neural Information Processing Systems 33: Annual Conference on Neural Information Processing Systems 2020, NeurIPS 2020, December 6-12, 2020, virtual.**  

6. **Jerry Chee, Yaohui Cai, Volodymyr Kuleshov, and Christopher De Sa. 2023.**  
   _QuIP: 2-bit quantization of large language models with guarantees._  
   **In Thirty-seventh Conference on Neural Information Processing Systems.**  

7. **Jack Choquette, Wishwesh Gandhi, Olivier Giroux, Nick Stam, and Ronny Krashinsky. 2021.**  
   _NVIDIA A100 tensor core GPU: performance and innovation._  
   **IEEE Micro, 41(2):29–35.**  

8. **Hyung Won Chung, Le Hou, Shayne Longpre, Barret Zoph, Yi Tay, William Fedus, Yunxuan Li, Xuezhi Wang, Mostafa Dehghani, Siddhartha Brahma, Albert Webson, Shixiang Shane Gu, Zhuyun Dai, Mirac Suzgun, Xinyun Chen, Aakanksha Chowdhery, Alex Castro-Ros, Marie Pellat, Kevin Robinson, Dasha Valter, Sharan Narang, Gaurav Mishra, Adams Yu, Vincent Zhao, Yanping Huang, Andrew Dai, Hongkun Yu, Slav Petrov, Ed H. Chi, Jeff Dean, Jacob Devlin, Adam Roberts, Denny Zhou, Quoc V. Le, and Jason Wei. 2024.**  
   _Scaling instruction-finetuned language models._  
   **Journal of Machine Learning Research, 25(70):1–53.**  

9. **Karl Cobbe, Vineet Kosaraju, Mohammad Bavarian, Mark Chen, Heewoo Jun, Lukasz Kaiser, Matthias Plappert, Jerry Tworek, Jacob Hilton, Reiichiro Nakano, Christopher Hesse, and John Schulman. 2021.**  
   _Training verifiers to solve math word problems._  
   **CoRR, abs/2110.14168.**  

10. **Tim Dettmers, Mike Lewis, Younes Belkada, and Luke Zettlemoyer. 2022.**  
    _Gpt3.int8(): 8-bit matrix multiplication for transformers at scale._  
    **In Advances in Neural Information Processing Systems 35: Annual Conference on Neural Information Processing Systems 2022, NeurIPS 2022, New Orleans, LA, USA, November 28 - December 9, 2022.**  

11. **Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, and Luke Zettlemoyer. 2023.**  
    _Qlora: Efficient finetuning of quantized llms._  
    **In Advances in Neural Information Processing Systems 36: Annual Conference on Neural Information Processing Systems 2023, NeurIPS 2023, New Orleans, LA, USA, December 10 - 16, 2023.**  

12. **Tim Dettmers, Ruslan A. Svirschevski, Vage Egiazarian, Denis Kuznedelev, Elias Frantar, Saleh Ashkboos, Alexander Borzunov, Torsten Hoefler, and Dan Alistarh. 2024.**  
    _SpQR: A sparse-quantized representation for near-lossless LLM weight compression._  
    **In The Twelfth International Conference on Learning Representations.**  

13. **Qingxiu Dong, Lei Li, Damai Dai, Ce Zheng, Zhiyong Wu, Baobao Chang, Xu Sun, Jingjing Xu, Lei Li, and Zhifang Sui. 2023.**  
    _A survey for in-context learning._  
    **CoRR, abs/2301.00234.**  

14. **Dayou Du, Yijia Zhang, Shijie Cao, Jiaqi Guo, Ting Cao, Xiaowen Chu, and Ningyi Xu. 2024.**  
    _Bitdistiller: Unleashing the potential of sub-4-bit llms via self-distillation._  
    **CoRR, abs/2402.10631.**  

15. **Chelsea Finn, Pieter Abbeel, and Sergey Levine. 2017.**  
    _Model-agnostic meta-learning for fast adaptation of deep networks._  
    **In Proceedings of the 34th International Conference on Machine Learning, ICML 2017, Sydney, NSW, Australia, 6-11 August 2017,** volume 70 of **Proceedings of Machine Learning Research,** pages 1126–1135. PMLR.  

16. **Jonathan Frankle and Michael Carbin. 2019.**  
    _The lottery ticket hypothesis: Finding sparse, trainable neural networks._  
    **In 7th International Conference on Learning Representations, ICLR 2019, New Orleans, LA, USA, May 6-9, 2019.** OpenReview.net.  

17. **Elias Frantar and Dan Alistarh. 2022.**  
    _Optimal brain compression: A framework for accurate post-training quantization and pruning._  
    **In Advances in Neural Information Processing Systems.**  

18. **Elias Frantar and Dan Alistarh. 2023.**  
    _Sparsegpt: Massive language models can be accurately pruned in one-shot._  
    **In International Conference on Machine Learning, ICML 2023, 23-29 July 2023, Honolulu, Hawaii, USA,** volume 202 of **Proceedings of Machine Learning Research,** pages 10323–10337. PMLR.  

19. **Elias Frantar, Saleh Ashkboos, Torsten Hoefler, and Dan Alistarh. 2023.**  
    _OPTQ: Accurate quantization for generative pre-trained transformers._  
    **In The Eleventh International Conference on Learning Representations.**   

20. **Yao Fu, Hao Peng, Litu Ou, Ashish Sabharwal, and Tushar Khot. 2023.**  
    _Specializing smaller language models towards multi-step reasoning._  
    **In Proceedings of the 40th International Conference on Machine Learning,** volume 202 of **Proceedings of Machine Learning Research,** pages 10421–10430. PMLR.  

21. **Leo Gao, Jonathan Tow, Baber Abbasi, Stella Biderman, Sid Black, Anthony DiPofi, Charles Foster, Laurence Golding, Jeffrey Hsu, Alain Le Noac’h, Haonan Li, Kyle McDonell, Niklas Muennighoff, Chris Ociepa, Jason Phang, Laria Reynolds, Hailey Schoelkopf, Aviya Skowron, Lintang Sutawika, Eric Tang, Anish Tithe, Ben Wang, Kevin Wang, and Andy Zou. 2023.**  
    _A framework for few-shot language model evaluation._  

22. **Mor Geva, Daniel Khashabi, Elad Segal, Tushar Khot, Dan Roth, and Jonathan Berant. 2021.**  
    _Did Aristotle use a laptop? A question answering benchmark with implicit reasoning strategies._  
    **Trans. Assoc. Comput. Linguistics, 9:** 346–361.  

23. **R.M. Gray and D.L. Neuhoff. 1998.**  
    _Quantization._  
    **IEEE Transactions on Information Theory, 44(6):2325–2383.**  

24. **Yuxian Gu, Li Dong, Furu Wei, and Minlie Huang. 2024.**  
    _MiniLLM: Knowledge distillation of large language models._  
    **In The Twelfth International Conference on Learning Representations.**  

25. **Cong Guo, Jiaming Tang, Weiming Hu, Jingwen Leng, Chen Zhang, Fan Yang, Yunxin Liu, Minyi Guo, and Yuhao Zhu. 2023.**  
    _Olive: Accelerating large language models via hardware-friendly outlier-victim pair quantization._  
    **In Proceedings of the 50th Annual International Symposium on Computer Architecture, ISCA 2023, Orlando, FL, USA, June 17-21, 2023,** pages 3:1–3:15. ACM.  

26. **Song Han, Huizi Mao, and William J. Dally. 2016.**  
    _Deep compression: Compressing deep neural network with pruning, trained quantization and huffman coding._  
    **In 4th International Conference on Learning Representations, ICLR 2016, San Juan, Puerto Rico, May 2-4, 2016, Conference Track Proceedings.**  

27. **Song Han, Jeff Pool, John Tran, and William Dally. 2015.**  
    _Learning both weights and connections for efficient neural network._  
    **In Advances in Neural Information Processing Systems,** volume 28. Curran Associates, Inc.  

28. **Geoffrey E. Hinton, Oriol Vinyals, and Jeffrey Dean. 2015.**  
    _Distilling the knowledge in a neural network._  
    **CoRR, abs/1503.02531.**  

29. **Namgyu Ho, Laura Schmid, and Se-Young Yun. 2023.**  
    _Large language models are reasoning teachers._  
    **In Proceedings of the 61st Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 14852–14882. Association for Computational Linguistics.  

30. **Coleman Hooper, Sehoon Kim, Hiva Mohammadzadeh, Michael W. Mahoney, Yakun Sophia Shao, Kurt Keutzer, and Amir Gholami. 2024.**  
    _Kvquant: Towards 10 million context length LLM inference with KV cache quantization._  
    **CoRR, abs/2401.18079.**  

31. **Cheng-Yu Hsieh, Chun-Liang Li, Chih-Kuan Yeh, Hootan Nakhost, Yasuhisa Fujii, Alex Ratner, Ranjay Krishna, Chen-Yu Lee, and Tomas Pfister. 2023.**  
    _Distilling step-by-step! outperforming larger language models with less training data and smaller model sizes._  
    **In Findings of the Association for Computational Linguistics: ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 8003–8017. Association for Computational Linguistics.  

32. **Yukun Huang, Yanda Chen, Zhou Yu, and Kathleen R. McKeown. 2022.**  
    _In-context learning distillation: Transferring few-shot learning ability of pre-trained language models._  
    **CoRR, abs/2212.10670.**  

33. **Hyesung Jeon, Yulhwa Kim, and Jae-Joon Kim. 2024.**  
    _L4Q: parameter efficient quantization-aware training on large language models via lora-wise LSQ._  
    **CoRR, abs/2402.04902.**  

34. **Yuxin Jiang, Chunkit Chan, Mingyang Chen, and Wei Wang. 2023.**  
    _Lion: Adversarial distillation of proprietary large language models._  
    **In Proceedings of the 2023 Conference on Empirical Methods in Natural Language Processing,** pages 3134–3154, Singapore. Association for Computational Linguistics.       

35. **Jared Kaplan, Sam McCandlish, Tom Henighan, Tom B. Brown, Benjamin Chess, Rewon Child, Scott Gray, Alec Radford, Jeffrey Wu, and Dario Amodei. 2020.**  
   _Scaling laws for neural language models._  
   **CoRR, abs/2001.08361.**  

36. **Bo-Kyeong Kim, Geonmin Kim, Tae-Ho Kim, Thibault Castells, Shinkook Choi, Junho Shin, and Hyoung-Kyu Song. 2024.**  
   _Shortened llama: A simple depth pruning for large language models._  
   **ICLR Workshop on Mathematical and Empirical Understanding of Foundation Models (ME-FoMo).**  

37. **Jeonghoon Kim, Jung Hyun Lee, Sungdong Kim, Joonsuk Park, Kang Min Yoo, Se Jung Kwon, and Dongsoo Lee. 2023a.**  
   _Memory-efficient fine-tuning of compressed large language models via sub-4-bit integer quantization._  
   **In Thirty-seventh Conference on Neural Information Processing Systems.**  

38. **Sehoon Kim, Coleman Hooper, Amir Gholami, Zhen Dong, Xiuyu Li, Sheng Shen, Michael W. Mahoney, and Kurt Keutzer. 2023b.**  
   _Squeezellm: Dense-and-sparse quantization._  
   **CoRR, abs/2306.07629.**  

39. **Yann LeCun, John S. Denker, and Sara A. Solla. 1989.**  
   _Optimal brain damage._  
   **In Advances in Neural Information Processing Systems 2, [NIPS Conference, Denver, Colorado, USA, November 27-30, 1989],** pages 598–605. Morgan Kaufmann.  

40. **Changhun Lee, Jungyu Jin, Taesu Kim, Hyungjun Kim, and Eunhyeok Park. 2024.**  
   _OWQ: outlier-aware weight quantization for efficient fine-tuning and inference of large language models._  
   **In Thirty-Eighth AAAI Conference on Artificial Intelligence, AAAI 2024, Thirty-Sixth Conference on Innovative Applications of Artificial Intelligence, IAAI 2024, Fourteenth Symposium on Educational Advances in Artificial Intelligence, EAAI 2014, February 20-27, 2024, Vancouver, Canada,** pages 13355–13364. AAAI Press.  

41. **Ming Li, Lichang Chen, Jiuhai Chen, Shwai He, Jiuxiang Gu, and Tianyi Zhou. 2024a.**  
   _Selective reflection-tuning: Student-selected data recycling for LLM instruction-tuning._  
   **CoRR, abs/2402.10110.**  

42. **Shiyang Li, Jianshu Chen, Yelong Shen, Zhiyu Chen, Xinliu Zhang, Zekun Li, Hong Wang, Jing Qian, Baolin Peng, Yi Mao, Wenhu Chen, and Xifeng Yan. 2024b.**  
   _Explanations from large language models make small reasoners better._  
   **In 2nd Workshop on Sustainable AI.**  

43. **Yiwei Li, Peiwen Yuan, Shaoxiong Feng, Boyuan Pan, Bin Sun, Xinglin Wang, Heda Wang, and Kan Li. 2024c.**  
   _Turning dust into gold: Distilling complex reasoning capabilities from llms by leveraging negative data._  
   **In Thirty-Eighth AAAI Conference on Artificial Intelligence, AAAI 2024, Thirty-Sixth Conference on Innovative Applications of Artificial Intelligence, IAAI 2024, Fourteenth Symposium on Educational Advances in Artificial Intelligence, EAAI 2014, February 20-27, 2024, Vancouver, Canada,** pages 18591–18599. AAAI Press.  

44. **Yixiao Li, Yifan Yu, Chen Liang, Pengcheng He, Nikos Karampatziakis, Weizhu Chen, and Tuo Zhao. 2023a.**  
   _Loftq: Lora-fine-tuning-aware quantization for large language models._  
   **CoRR, abs/2310.08659.**  

45. **Yun Li, Lin Niu, Xipeng Zhang, Kai Liu, Jianchen Zhu, and Zhanhui Kang. 2023b.**  
   _E-sparse: Boosting the large language model inference through entropy-based N: M sparsity._  
   **CoRR, abs/2310.15929.**  

46. **Zhuo Li, Hengyi Li, and Lin Meng. 2023c.**  
   _Model compression for deep neural networks: A survey._  
   **Comput., 12(3):60.**  

47. **Chen Liang, Simiao Zuo, Qingru Zhang, Pengcheng He, Weizhu Chen, and Tuo Zhao. 2023.**  
   _Less is more: Task-aware layer-wise distillation for language model compression._  
   **In International Conference on Machine Learning, ICML 2023, 23-29 July 2023, Honolulu, Hawaii, USA,** volume 202 of **Proceedings of Machine Learning Research,** pages 20852–20867. PMLR.  

48. **Ji Lin, Jiaming Tang, Haotian Tang, Shang Yang, Xingyu Dang, and Song Han. 2023.**  
   _AWQ: activation-aware weight quantization for LLM compression and acceleration._  
   **CoRR, abs/2306.00978.**  

49. **Shih-yang Liu, Zechun Liu, Xijie Huang, Pingcheng Dong, and Kwang-Ting Cheng. 2023a.**  
   _LLM-FP4: 4-bit floating-point quantized transformers._  
   **In Proceedings of the 2023 Conference on Empirical Methods in Natural Language Processing,** pages 592–605, Singapore. Association for Computational Linguistics.  

50. **Yuxuan Liu. 2024.**  
   _Learning to reason with autoregressive in-context distillation._  
   **In The Second Tiny Papers Track at ICLR 2024.**  

51. **Zechun Liu, Barlas Oguz, Changsheng Zhao, Ernie Chang, Pierre Stock, Yashar Mehdad, Yangyang Shi, Raghuraman Krishnamoorthi, and Vikas Chandra. 2023b.**  
   _LLM-QAT: data-free quantization aware training for large language models._  
   **CoRR, abs/2305.17888.**  

52. **Zirui Liu, Jiayi Yuan, Hongye Jin, Shaochen Zhong, Zhaozhuo Xu, Vladimir Braverman, Beidi Chen, and Xia Hu. 2024.**  
   _KIVI: A tuning-free asymmetric 2bit quantization for KV cache._  
   **CoRR, abs/2402.02750.**  

53. **Xinyin Ma, Gongfan Fang, and Xinchao Wang. 2023.**  
   _LLM-pruner: On the structural pruning of large language models._  
   **In Thirty-seventh Conference on Neural Information Processing Systems.**  

54. **Lucie Charlotte Magister, Jonathan Mallinson, Jakub Adámek, Eric Malmi, and Aliaksei Severyn. 2023.**  
   _Teaching small language models to reason._  
   **In Proceedings of the 61st Annual Meeting of the Association for Computational Linguistics (Volume 2: Short Papers), ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 1773–1781. Association for Computational Linguistics.  

55. **Mitchell P. Marcus, Beatrice Santorini, and Mary Ann Marcinkiewicz. 1993.**  
   _Building a large annotated corpus of English: The Penn Treebank._  
   **Computational Linguistics, 19(2):313–330.**  

56. **Colin Raffel, Noam Shazeer, Adam Roberts, Katherine Lee, Sharan Narang, Michael Matena, Yanqi Zhou, Wei Li, and Peter J. Liu. 2020.**  
   _Exploring the limits of transfer learning with a unified text-to-text transformer._  
   **J. Mach. Learn. Res., 21:140:1–140:67.**  

57. **Mohammad Rastegari, Vicente Ordonez, Joseph Redmon, and Ali Farhadi. 2016.**  
   _Xnor-net: ImageNet classification using binary convolutional neural networks._  
   **In Computer Vision - ECCV 2016 - 14th European Conference, Amsterdam, The Netherlands, October 11-14, 2016, Proceedings, Part IV,** volume 9908 of **Lecture Notes in Computer Science,** pages 525–542. Springer.  

58. **Anna Rogers, Olga Kovaleva, and Anna Rumshisky. 2020.**  
   _A primer in BERTology: What we know about how BERT works._  
   **Transactions of the Association for Computational Linguistics, 8:** 842–866.  

59. **Rajarshi Saha, Varun Srivastava, and Mert Pilanci. 2023.**  
   _Matrix compression via randomized low rank and low precision factorization._  
   **In Advances in Neural Information Processing Systems 36: Annual Conference on Neural Information Processing Systems 2023, NeurIPS 2023, New Orleans, LA, USA, December 10 - 16, 2023.**  

60. **Pedro Savarese and Michael Maire. 2019.**  
   _Learning implicitly recurrent CNNs through parameter sharing._  
   **In 7th International Conference on Learning Representations, ICLR 2019, New Orleans, LA, USA, May 6-9, 2019.** OpenReview.net.  

61. **Teven Le Scao, Angela Fan, Christopher Akiki, Ellie Pavlick, Suzana Ilic, Daniel Hesslow, Roman Castagné, Alexandra Sasha Luccioni, François Yvon, Matthias Gallé, Jonathan Tow, Alexander M. Rush, Stella Biderman, Albert Webson, Pawon Sasanka Bamanamanchi, Thomas Wang, Benoît Sagot, Niklas Muennighoff, Albert Villanova del Moral, Olatunji Ruwase, Rachel Bawden, Stas Bekman, Angelica McMillan-Major, Iz Beltagy, Huu Nguyen, Lucile Saulnier, Samson Tan, Pedro Ortiz Suarez, Victor Sanh, Hugo Laurençon, Yacine Jernite, Julien Launay, Marcin Kardas, Tim Dettmers, and others.**  
   _BLOOM: A 176b-parameter open-access multilingual language model._  
   **CoRR, abs/2211.05100.**  

62. **Adi Simhi, Aitor Soroa, Ahlam Fikri Aji, Amit Alfassy, Anna Rogers, Ariel Kreisberg Nitzav, Canwen Xu, Chenghao Mou, Chris Emezue, Christopher Klamm, Colin Leong, Daniel van Strien, David Ifeoluwa Adelani, and others. 2022.**  
   **BLOOM: A 176b-parameter open-access multilingual language model.**  
   **CoRR, abs/2211.05100.**  

63. **Hang Shao, Bei Liu, and Yanmin Qian. 2024a.**  
   _One-shot sensitivity-aware mixed sparsity pruning for large language models._  
   **In ICASSP 2024 - 2024 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP),** pages 11296–11300.  

64. **Wengqi Shao, Mengzhao Chen, Zhaoyang Zhang, Peng Xu, Lirui Zhao, Zhiqian Li, Kaipeng Zhang, Peng Gao, Yu Qiao, and Ping Luo. 2024b.**  
   _Omniquant: Omnidirectionally calibrated quantization for large language models._  
   **In The Twelfth International Conference on Learning Representations.**  

65. **Pratyusha Sharma, Jordan T. Ash, and Dipendra Misra. 2024.**  
   _The truth is in there: Improving reasoning with layer-selective rank reduction._  
   **In The Twelfth International Conference on Learning Representations.**  

66. **Kumar Shridhar, Alessandro Stolfo, and Mrinmaya Sachan. 2023.**  
   _Distilling reasoning capabilities into smaller language models._  
   **In Findings of the Association for Computational Linguistics: ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 7059–7073. Association for Computational Linguistics.  

67. **Nathan Srebro and Tommi S. Jaakkola. 2003.**  
   _Weighted low-rank approximations._  
   **In Machine Learning, Proceedings of the Twentieth International Conference (ICML 2003), August 21-24, 2003, Washington, DC, USA,** pages 720–727. AAAI Press.  

68. **Aaron Srivastava, Abhinav Rastogi, Abhishek Rao, Abu Awal Md Shoeb, Abubakar Abid, Adam Fisch, Adam R. Brown, Adam Santoro, Aditya Gupta, Adria Garriga-Alonso, and others.**  
   **Training language models with human feedback.**  
   **In NeurIPS.**  

69. **Hayden Bogar, Henry Francis Anthony Shevlin, Hinrich Schuetze, Hiromu Yakura, Hongming Zhang, Hugh Mee Wong, Ian Ng, Isaac Noble, Jaap Jumelet, Jack Geissinger, Jackson Kernion, Jacob Hilton, Jaehoon Lee, Jaime Fernández Fisac, and others.**  
   **Beyond the imitation game: Quantifying and extrapolating the capabilities of language models.**  
   **Transactions on Machine Learning Research.**  

70. **Samuel Stanton, Pavel Izmailov, Polina Kirichenko, Alexander A. Alemi, and Andrew Gordon Wilson. 2021.**  
   _Does knowledge distillation really work?_  
   **In Advances in Neural Information Processing Systems 34: Annual Conference on Neural Information Processing Systems 2021, NeurIPS 2021, December 6-14, 2021, virtual,** pages 6906–6919.  

71. **Mingjie Sun, Zhuang Liu, Anna Bair, and J Zico Kolter. 2024.**  
   _A simple and effective pruning approach for large language models._  
   **In The Twelfth International Conference on Learning Representations.**  

72. **Alon Talmor, Jonathan Herzig, Nicholas Lourie, and Jonathan Berant. 2019.**  
   _CommonsenseQA: A question answering challenge targeting commonsense knowledge._  
   **In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers),** pages 4149–4158, Minneapolis, Minnesota. Association for Computational Linguistics.  

73. **Sandeep Tata and Jignesh M. Patel. 2003.**  
   _Piq: An algebra for querying protein data sets._  
   **In Proceedings of the 15th International Conference on Scientific and Statistical Database Management (SSDBM 2003), 9-11 July 2003, Cambridge, MA, USA,** pages 141–150. IEEE Computer Society.  

74. **Hugo Touvron, Thibaut Lavril, Gautier Izacard, Xavier Martinet, Marie-Anne Lachaux, Timothée Lacroix, Baptiste Rozière, Naman Goyal, Eric Hambro, Faisal Azhar, Aurélien Rodriguez, Armand Joulin, Edouard Grave, and Guillaume Lample. 2023a.**  
   _Llama: Open and efficient foundation language models._  
   **CoRR, abs/2302.13971.**      

75. **Hugo Touvron, Louis Martin, Kevin Stone, Peter Albert, Amjad Almahairi, Yasmine Babaei, Nikolay Bashlykov, Soumya Patra, Prajwal Bhargava, Shruti Bhosale, Dan Bikel, Lukas Blecher, Cristian Canton-Ferrer, Moya Chen, Guillem Cucurull, David Esiobu, Jude Fernandes, Jeremy Fu, Wenyin Fu, Brian Fuller, Cynthia Gao, Vedanuj Goswami, Naman Goyal, Anthony Hartshorn, Saghar Hosseini, Rui Hou, Hakan Inan, Marcin Kardas, Viktor Kerkez, Madian Khabsa, Isabel Kloumann, Artem Korenev, Punit Singh Koura, Marie-Anne Lachaux, Thibaut Lavril, Jenya Lee, Diana Liskovich, Yinghai Lu, Yuning Mao, Xavier Martinet, Todor Mihaylov, Pushkar Mishra, Igor Molybog, Yixin Nie, Andrew Poulton, Jeremy Reizenstein, Rashi Rungta, Kalyan Saladi, Alan Schelten, Ruan Silva, Eric Michael Smith, Ranjan Subramanian, Xiaoqing Ellen Tan, Binh Tang, Ross Taylor, Adina Williams, Jian Xiang Kuan, Puxin Xu, Zheng Yan, Iliyuan Zarov, Yuchen Zhang, Angela Fan, Melanie Kambadur, Sharan Narang, Aurélien Rodriguez, Robert Stojnic, Sergey Edunov, and Thomas Scialom. 2023b.**  
   _Llama 2: Open foundation and fine-tuned chat models._  
   **CoRR, abs/2307.09288.**  

76. **Ben Wang and Aran Komatsuzaki. 2021.**  
   _GPT-J-6B: A 6 Billion Parameter Autoregressive Language Model._  
   **https://github.com/kingoflolz/mesh-transformer-jax.**  

77. **Peifeng Wang, Zhengyang Wang, Zheng Li, Yifan Gao, Bing Yin, and Xiang Ren. 2023a.**  
   _SCOTT: self-consistent chain-of-thought distillation._  
   **In Proceedings of the 61st Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 5546–5558. Association for Computational Linguistics.  

78. **Xinyi Wang, Wanrong Zhu, Michael Saxon, Mark Steyvers, and William Yang Wang. 2023b.**  
   _Large language models are latent variable models: Explaining and finding good demonstrations for in-context learning._  
   **In Thirty-seventh Conference on Neural Information Processing Systems.**  

79. **Xuezhi Wang, Jason Wei, Dale Schuurmans, Quoc V. Le, Ed H. Chi, Sharan Narang, and others. 2022.**  
   _Chain-of-thought prompting elicits reasoning in large language models._  
   **In NeurIPS.**  

80. **Xiuying Wei, Yunchen Zhang, Yuhang Li, Xiangguo Zhang, Ruihao Gong, Jinyang Guo, and Xianglong Liu. 2023.**  
   _Outlier suppression+: Accurate quantization of large language models by equivalent and effective shifting and scaling._  
   **In Proceedings of the 2023 Conference on Empirical Methods in Natural Language Processing,** pages 1648–1665, Singapore. Association for Computational Linguistics.  

81. **Wei Wen, Chunpeng Wu, Yandan Wang, Yiran Chen, and Hai Li. 2016.**  
   _Learning structured sparsity in deep neural networks._  
   **In Advances in Neural Information Processing Systems,** volume 29. Curran Associates, Inc.  

82. **Miles Williams and Nikolaos Aletras. 2023.**  
   _How does calibration data affect the post-training pruning and quantization of large language models?_  
   **CoRR, abs/2311.09755.**  

83. **Samuel Williams, Andrew Waterman, and David A. Patterson. 2009.**  
   _Roofline: an insightful visual performance model for multicore architectures._  
   **Commun. ACM, 52(4):65–76.**  

84. **Minghao Wu, Abdul Waheed, Chiyu Zhang, Muhammad Abdul-Mageed, and Alham Aji. 2024.**  
   _LaMini-LM: A diverse herd of distilled models from large-scale instructions._  
   **In Proceedings of the 18th Conference of the European Chapter of the Association for Computational Linguistics (Volume 1: Long Papers),** pages 944–964, St. Julian’s, Malta. Association for Computational Linguistics.  

85. **Haojun Xia, Zhen Zheng, Yuchao Li, Donglin Zhuang, Zhongzhu Zhou, Xiafei Qiu, Yong Li, Wei Lin, and Shuaiwen Leon Song. 2023.**  
   _Flash-llm: Enabling cost-effective and highly-efficient large generative model inference with unstructured sparsity._  
   **Proc. VLDB Endow., 17(2):211–224.**  

86. **Mengzhou Xia, Tianyu Gao, Zhiyuan Zeng, and Danqi Chen. 2024.**  
   _Sheared LLaMA: Accelerating language model pre-training via structured pruning._  
   **In The Twelfth International Conference on Learning Representations.**  

87. **Aakanksha Chowdhery, and Denny Zhou. 2023c.**  
   _Self-consistency improves chain of thought reasoning in language models._  
   **In The Eleventh International Conference on Learning Representations, ICLR 2023, Kigali, Rwanda, May 1-5, 2023.** OpenReview.net.  

88. **Yizhong Wang, Yeghana Kordi, Swaroop Mishra, Alisa Liu, Noah A. Smith, Daniel Khashabi, and Hannaneh Hajishirzi. 2023d.**  
   _Self-instruct: Aligning language models with self-generated instructions._  
   **In Proceedings of the 61st Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 13484–13508. Association for Computational Linguistics.  

89. **Yizhong Wang, Yeghana Kordi, Swaroop Mishra, Alisa Liu, Noah A. Smith, Daniel Khashabi, and Hannaneh Hajishirzi. 2023e.**  
   _Self-instruct: Aligning language models with self-generated instructions._  
   **In Proceedings of the 61st Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), ACL 2023, Toronto, Canada, July 9-14, 2023,** pages 13484–13508. Association for Computational Linguistics.  

90. **Yue Wang, Weishi Wang, Shafiq R. Joty, and Steven C. H. Hoi. 2021.**  
   _Codet5: Identifier-aware unified pre-trained encoder-decoder models for code understanding and generation._  
   **In Proceedings of the 2021 Conference on Empirical Methods in Natural Language Processing, EMNLP 2021, Virtual Event / Punta Cana, Dominican Republic, 7-11 November, 2021,** pages 8696–8708. Association for Computational Linguistics.  

91. **Zhaoyang Wang, Shaohan Huang, Yuxuan Liu, Jiahai Wang, Minghui Song, Zihan Zhang, Haizhen Huang, Furu Wei, Weiwei Deng, Feng Sun, and Qi Zhang. 2023f.**  
   _Democratizing reasoning ability: Tailored learning from large language model._  
   **In Proceedings of the 2023 Conference on Empirical Methods in Natural Language Processing, EMNLP 2023, Singapore, December 6-10, 2023,** pages 1948–1966. Association for Computational Linguistics.  

92. **Zhihang Yuan, Yuzhang Shang, Yue Song, Qiang Wu, Yan Yan, and Guangyu Sun. 2023b.**  
   _ASVD: activation-aware singular value decomposition for compressing large language models._  
   **CoRR, abs/2312.05821.**  

93. **Yuxuan Yue, Zhihang Yuan, Haojie Duanmu, Sifan Zhou, Jianlong Wu, and Liqiang Nie. 2024.**  
   _Wkvquant: Quantizing weight and key/value cache for large language models gains more._  
   **CoRR, abs/2402.12065.**  

94. **Susan Zhang, Stephen Roller, Naman Goyal, Mikel Artetxe, Moya Chen, Shuohui Chen, Christopher Dewan, Mona T. Diab, Xian Li, Xi Victoria Lin, Todor Mihaylov, Myle Ott, Sam Shleifer, Kurt Shuster, Daniel Simig, Punit Singh Koura, Anjali Sridhar, Tianlu Wang, and Luke Zettlemoyer. 2022.**  
   _OPT: open pre-trained transformer language models._  
   **CoRR, abs/2205.01068.**  

95. **Yuxin Zhang, Lirui Zhao, Mingbao Lin, Sun Yunyun, Yiwu Yao, Xingjia Han, Jared Tanner, Shiwei Liu, and Rongrong Ji. 2024.**  
   _Dynamic sparse no training: Training-free fine-tuning for sparse LLMs._  
   **In The Twelfth International Conference on Learning Representations.**  

96. **Wayne Xin Zhao, Kun Zhou, Junyi Li, Tianyi Tang, Xiaolei Wang, Yupeng Hou, Yingqian Min, Beichen Zhang, Junjie Zhang, Zican Dong, Yifan Du, Chen Yang, Yushuo Chen, Zhipeng Chen, Jinhao Jiang, Ruiyang Ren, Yifan Li, Xinyu Tang, Zikang Liu, Peiyu Liu, Jian-Yun Nie, and Ji-Rong Wen. 2023.**  
   _A survey of large language models._  
   **CoRR, abs/2303.18223.**  

97. **Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang, Zi Lin, Zhuohan Li, Dacheng Li, Eric P. Xing, Hao Zhang, Joseph E. Gonzalez, and Ion Stoica. 2023.**  
   _Judging llm-as-a-judge with mt-bench and chatbot arena._  
   **In Advances in Neural Information Processing Systems 36: Annual Conference on Neural Information Processing Systems 2023, NeurIPS 2023, New Orleans, LA, USA, December 10 - 16, 2023.**  

98. **Xuekai Zhu, Biqing Qi, Kaiyan Zhang, Xinwei Long, Zhouhan Lin, and Bowen Zhou. 2024.**  
   _PaD: Program-aided distillation can teach small models reasoning better than chain-of-thought fine-tuning._  
   **In Proceedings of the 2024 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies (Volume 1: Long Papers),** pages 2571–2597, Mexico City, Mexico. Association for Computational Linguistics.  

99. **Barret Zoph and Quoc V. Le. 2017.**  
   _Neural architecture search with reinforcement learning._  
   **In 5th International Conference on Learning Representations, ICLR 2017, Toulon, France, April 24-26, 2017, Conference Track Proceedings.** OpenReview.net.     